In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score

    
import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\data job posts.csv
input\screenshot.jpg


In [8]:
data = pd.read_csv('input/data job posts.csv')
print(data.columns)
# selecting only IT Jobs
df = data[data['IT']]
# selecting 
cols = ['RequiredQual', 'Eligibility', 'Title', 'JobDescription', 'JobRequirment']
df=df[cols]
df.head(5)

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')


,RequiredQual,Eligibility,Title,JobDescription,JobRequirment
4,- University degree; economical background is ...,NaN,Software Developer,NaN,- Rendering technical assistance to Database M...
15,"- Excellent knowledge of Windows 2000 Server, ...",NaN,Network Administrator,NaN,- Network monitoring and administration;\r\n- ...
19,"As a GD you are creative, innovative and have\...",NaN,Graphic Designer,The position of Graphic Designer (GD) demands ...,Graphic Designer will be responsible for every...
34,Participants should be mid-level professionals...,NaN,Demographic Analysis Workshop,NaN,NaN
35,- Work experience of at least two years; \r\n-...,NaN,Programmer,NaN,NaN


# Editing Job Titles

In [9]:
classes = df['Title'].value_counts()[:21]
keys = classes.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

Software Developer           134
Web Developer                101
Java Developer                88
Graphic Designer              75
Software Engineer             69
Senior Java Developer         69
PHP Developer                 65
Senior Software Engineer      63
Programmer                    56
IT Specialist                 55
Senior QA Engineer            43
Senior Software Developer     41
Android Developer             37
.NET Developer                36
Senior .NET Developer         34
Senior PHP Developer          34
iOS Developer                 31
Software QA Engineer          29
Senior Web Developer          29
Database Developer            29
Database Administrator        28
Name: Title, dtype: int64

In [10]:
def chane_titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'

    else:
        return x
        
    
df['Title'] = df['Title'].apply(chane_titles)
df['Title'].value_counts()

Java Developer          157
Software Developer      134
Software Engineer       132
Web Developer           130
PHP Developer            99
Graphic Designer         75
Software QA Engineer     72
.NET Developer           70
Database Admin/Dev       57
Programmer               56
IT Specialist            55
Senior Web Developer     41
Android Developer        37
iOS Developer            31
Name: Title, dtype: int64

# Building tokenizer to process text

In [11]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        # lemmatize text - convert to base form 
        self.wnl = WordNetLemmatizer()
        # creating stopwords list, to ignore lemmatizing stopwords 
        self.stopwords = stopwords.words('english')
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.stopwords]

# removing new line characters, and certain hypen patterns                  
df['RequiredQual']=df['RequiredQual'].apply(lambda x: x.replace('\n', ' ').replace('\r', '').replace('- ', ''). replace(' - ', ' to '))

# Featurizing Text

In [12]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# train features and labels 
y = df['Title']
X = df['RequiredQual']
# tdif feature rep 
vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english')
vectorizer.fit(X)
# transoforming text to tdif features
tfidf_matrix = vectorizer.transform(X)
# sparse matrix to dense matrix for training
X_tdif = tfidf_matrix.toarray()
# encoding text labels in categories 
enc = LabelEncoder() 
enc.fit(y.values)
y_enc=enc.transform(y.values)

X_train_words, X_test_words, y_train, y_test = train_test_split(X, y_enc, test_size=0.15, random_state=10)

X_train = vectorizer.transform(X_train_words)
X_train = X_train.toarray()

X_test = vectorizer.transform(X_test_words)
X_test = X_test.toarray()

C:\Users\laptop\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\laptop\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
RFC = RandomForestClassifier(n_estimators=10000,
                                 max_depth= 1200,
                                 random_state=42)
train_preds = RFC.fit(X_train, y_train).predict(X_train)
test_preds = RFC.predict(X_test)

print('Train acc: {0}'.format(accuracy_score(y_train, train_preds)))
print('Test acc: {0}'.format(accuracy_score(y_test, test_preds)))

Train acc: 0.9969199178644764
Test acc: 0.7906976744186046


In [4]:
import pickle
filename = 'Job_Recommendation.sav'
pickle.dump(RFC, open(filename, 'wb'))

In [5]:
loaded_model = pickle.load(open(filename, 'rb'))

C:\Users\laptop\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\laptop\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [14]:
test_preds = loaded_model.predict_proba(X_test)

In [15]:
from sklearn.metrics import top_k_accuracy_score

In [23]:
top_k_accuracy_score(y_test, test_preds, k=3)

0.936046511627907

In [25]:
pickle.dump(vectorizer, open("tfidf.pickle", "wb"))

array([ 0,  2,  9, 10,  4, 11,  2,  9, 11,  5,  6, 10, 10,  0,  0, 10,  9,
        5,  9, 10, 13,  5,  1,  9, 12,  0,  9, 12, 11,  5,  5,  5,  5, 10,
       13, 12, 10, 10,  2,  9, 10,  3, 10,  4, 12, 10, 12, 10, 12,  4, 10,
        5,  5, 10,  7, 10,  5,  7, 10,  5, 10,  6,  3,  3, 12, 12,  5,  7,
        7,  2,  4,  0,  8, 12,  5,  0,  6,  8,  3,  2,  6, 12,  9,  6,  5,
       10,  7,  2,  5,  3,  2,  8,  9, 11, 10, 12, 12, 12, 12, 12,  9,  6,
        0,  6,  4,  3,  2, 11, 10,  5, 13,  5,  6, 11, 12,  5,  9,  4,  3,
        9, 10,  5, 13,  3,  1,  5,  9,  8,  5,  6,  5,  7,  5,  0,  1,  7,
        9,  3,  9,  5,  5,  7, 12, 12,  0, 10, 13,  7,  0, 10,  9, 10,  6,
        2, 10,  5,  7,  1,  9, 12, 11, 10,  6,  5,  3, 10,  7,  6,  2, 12,
        0, 10,  5,  8, 11,  1,  0,  5, 12,  5,  4,  6,  0,  2,  0,  6,  5,
        5,  7,  2,  3, 10, 12, 12,  1, 11,  5,  3,  9, 10,  5,  6,  5,  6,
        2,  0,  6,  5,  1,  3, 12,  9,  6,  3,  8,  9,  6,  9,  5,  0,  5,
        5,  0, 11,  5,  6